# 1. Import các thư viện cần thiết


In [5]:
import warnings
import os
from tqdm import tqdm
from typing import List
import pandas as pd
from unidecode import unidecode
import plotly.express as px
import unicodedata
import math
import json
import numpy as np
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")


# from langdetect import detect
# from langdetect import DetectorFactory

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

# 2. Tiền xử lý dữ liệu thống kê từ video


Đọc dữ liệu thống kê video thành DataFrame.


In [6]:
# raw_df1 = pd.read_csv("data/old_users_amthuc.csv", low_memory=False)
# raw_df1.info()

In [7]:
# raw_df2 = pd.read_csv("data/new_users_amthuc_1_handmade.csv", low_memory=False)
# raw_df2.info()

In [8]:
# raw_df3 = pd.read_csv("data/new_usrs_amthuc_250319_0033.csv", low_memory=False)
# raw_df3.info()

In [9]:
video_info_df = pd.read_csv("data/final_raw_videos.csv", low_memory=False)
video_info_df.info()
# video_info_df = pd.concat([raw_df1, raw_df2, raw_df3], ignore_index=True)
# video_info_df.info()
# video_info_df.to_csv('data/final_raw_videos.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71260 entries, 0 to 71259
Columns: 174 entries, stats.diggCount to duetInfo.duetFromId
dtypes: float64(90), int64(1), object(83)
memory usage: 94.6+ MB


In [10]:
# save_df = video_info_df[~video_info_df['author.uniqueId'].isna()]
# save_list = list(save_df['author.uniqueId'].unique())
# len(save_list)


# def write_list_to_file(file_path, data_list):
#     try:
#         with open(file_path, "w", encoding="utf-8") as file:
#             for item in data_list:
#                 file.write(str(item) + "\n")
#         print(f"Successfully wrote {len(data_list)} items to {file_path}")
#     except Exception as e:
#         print(f"Error writing to file: {e}")


# write_list_to_file("data/lists/users_from_new_old_csv.txt", save_list)

Loại bỏ các cột có tỷ lệ thiếu dữ liệu lớn hơn 50%


In [11]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display first 5 rows
missing_ratio.head()

music.coverLarge                              1.0
user.commerceUserInfo.downLoadLink.android    1.0
video.encodeUserTag                           1.0
poi.country                                   1.0
poi.fatherPoiId                               1.0
dtype: float64

In [12]:
# Remove columns with missing ratios greater than 0.50
video_info_df = video_info_df.dropna(
    axis='columns', thresh=0.50 * len(video_info_df)
)

In [13]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display the missing ratio
for column, ratio in missing_ratio[:5].items():
    print(f"{column:50}:{ratio:8.2%}")

video.claInfo.originalLanguageInfo.languageID     :  30.39%
video.claInfo.originalLanguageInfo.languageCode   :  30.39%
video.claInfo.captionsType                        :  30.39%
video.claInfo.originalLanguageInfo.canTranslateRealTimeNoCheck:  30.39%
video.claInfo.originalLanguageInfo.language       :  30.39%


In [14]:
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71260 entries, 0 to 71259
Data columns (total 87 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   stats.diggCount                                                 71260 non-null  int64  
 1   CategoryType                                                    70996 non-null  float64
 2   author.commentSetting                                           70996 non-null  float64
 3   author.downloadSetting                                          70996 non-null  float64
 4   author.duetSetting                                              70996 non-null  float64
 5   author.ftc                                                      70996 non-null  object 
 6   author.id                                                       70996 non-null  float64
 7   author.isADVirtual                               

Vì các cột có tên bắt đầu với `stats.*` chứa cùng thông tin với các cột có tên bắt đầu với `statsV2.*`, nhưng không có thông tin về `repostCount` như `statsV2.*`. Nên ta sẽ loại bỏ các cột có tên bắt đầu với `stats.*`.


In [15]:
# Remove columns starting with "stats."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("stats.")]
]

# video_info_df.info()

Xóa các cột bắt đầu với `video.claInfo.originalLanguageInfo.*` vì chúng chứa thông tin không cần thiết.


In [16]:
# Remove columns starting with "video.claInfo.originalLanguageInfo."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("video.claInfo.originalLanguageInfo.")]
]

# video_info_df.info()

Chuyển timestamp sang datetime và chuyển sang giờ VN.


In [17]:
# Chuyển timestamp sang dạng datetime UTC
video_info_df["createTime"] = pd.to_datetime(
    video_info_df["createTime"], unit="s", utc=True)

# Chuyển sang múi giờ Việt Nam (UTC+7)
video_info_df["createTime"] = video_info_df["createTime"].dt.tz_convert(
    "Asia/Ho_Chi_Minh")

Tạo một cột chứa danh sách các `hashtag` được trích xuất từ mô tả video. Và tính số lượng hashtag trong mỗi video.


In [18]:
# Replace missing values in "desc" column with an empty string
video_info_df["desc"] = video_info_df["desc"].fillna("")
video_info_df["desc"] = video_info_df["desc"].astype(str)
video_info_df["desc"] = video_info_df["desc"].str.strip()

# Create a new column for the hashtags
# and the number of hashtags in each video
video_info_df["hashtags"] = [""] * len(video_info_df)
video_info_df["num_hashtags"] = [0] * len(video_info_df)

# Extract hashtags from the "desc" column
# and Get the number of hashtags in each video
for index in tqdm(range(len(video_info_df))):
    # Get the description of the video
    description = video_info_df.loc[index, "desc"].strip().lower()

    if description:
        # # Remove emojis
        # description = description.encode('ascii', 'ignore').decode('ascii')
        # Chuyển tiếng Việt có dấu thành không dấu
        description = unidecode(description)

        # Add a space before all "#" characters
        description = description.replace("#", " #")

        # Find all strings starting with "#" and followed by a word
        hashtags = [word[1:] for word in description.split()
                    if word.startswith("#")]

        # Extract hashtags from the description
        video_info_df.loc[index, "hashtags"] = ",".join(hashtags).strip()

        # Get the number of hashtags
        video_info_df.loc[index, "num_hashtags"] = len(hashtags)
    else:
        video_info_df.loc[index, "hashtags"] = ""
        video_info_df.loc[index, "num_hashtags"] = 0

100%|██████████| 71260/71260 [00:11<00:00, 6422.45it/s]


In [19]:
# Bước 1: Chuyển hashtags thành danh sách
video_info_df["hashtags"] = video_info_df["hashtags"].apply(
    lambda x: x.split(",") if x else [])

### Chọn ra một số cột để kiểm tra thông số.


In [20]:
selected_cols = {
    'id': 'id',
    'author.uniqueId': 'username',
    "statsV2.playCount": "views",  # Lượt xem video
    "statsV2.diggCount": "likes",  # Lượt thích video
    "statsV2.commentCount": "comments",  # Lượt bình luận video
    "statsV2.shareCount": "shares",  # Lượt chia sẻ video
    'statsV2.collectCount': "collects",
    'authorStats.diggCount': "user.likes",
    'authorStats.followerCount': "user.followers",
    'authorStats.followingCount': "user.followings",
    # 'authorStats.friendCount': "user.friends",
    'authorStats.heartCount': "user.hearts",
    'authorStats.videoCount': "user.videos",
    'hashtags': 'hashtags',
    'createTime': 'createTime',
    'desc': 'desc',
    'textLanguage': 'textLang'
}
stats_df = video_info_df[list(selected_cols.keys())].rename(
    columns=selected_cols)
stats_df.sample(5)

,id,username,views,likes,comments,shares,collects,user.likes,user.followers,user.followings,user.hearts,user.videos,hashtags,createTime,desc,textLang
7865,7.459767e+18,nguyenhimatto,1200000.0,42700.0,185.0,499.0,1545.0,1072.0,597900.0,127.0,27000000.0,803.0,"[nguyenhimatto, nguyentainguyen, mukbang, nguy...",2025-01-14 20:45:54+07:00,Trả lời @Thảo Vy Oke uy tín luôn nhá Vy ới 😂😂😂...,vi
55796,7.428144e+18,dajsytran.92,7774.0,80.0,21.0,11.0,22.0,18400.0,53500.0,195.0,1000000.0,171.0,"[laugalae, nauancungtiktok, thinhhanhxuhuong, ...",2024-10-21 15:31:09+07:00,Trời bắt đầu se lạnh với những cơn mưa bắt chợ...,vi
52546,7.447145e+18,ngaymaibanangi,90900.0,2192.0,16.0,724.0,675.0,1302.0,64100.0,0.0,1100000.0,623.0,"[xuhuong, saigonfood, ngaymaibanangi, quananng...",2024-12-11 20:25:51+07:00,Bánh chuối THÁI LAN#xuhuong #saigonfood #ngaym...,vi
54170,7.320597e+18,ansaphanoi,408000.0,8670.0,50.0,46.0,288.0,3171.0,1600000.0,1.0,46000000.0,848.0,"[ansaphanoi, vtmgr, thanhthoiluottet, learnont...",2024-01-05 19:54:03+07:00,Đi ăn nem nướng chứ không phải đi xin việc #an...,vi
62910,7.366594e+18,nglethuthuy,343600.0,9265.0,95.0,549.0,705.0,2783.0,325600.0,186.0,9900000.0,551.0,"[nglethuthuy, learnontiktok, reviewanngon, detox]",2024-05-08 18:48:31+07:00,thử detox 3 ngày giống các chị em và cái kết 💞...,vi


Drop các dòng có thiếu hơn 50% số cột.


In [21]:
stats_df["missing_count"] = stats_df.isnull().sum(axis=1)
# Xóa các dòng có hơn 50% giá trị bị thiếu
threshold = stats_df.shape[1] * 0.5

# Lọc bỏ các dòng có số lượng giá trị thiếu vượt quá ngưỡng
stats_df = stats_df[stats_df["missing_count"] <= threshold].drop(
    columns=["missing_count"])
stats_df.shape

(70996, 16)

# 3. Lọc lại danh sách user


In [22]:
print("Number of users: ", stats_df['username'].nunique())

Number of users:  264


Lọc các video đăng trong năm 2025 (loại bỏ người dùng không hoạt động).


In [23]:
# time_thres = pd.to_datetime("2025-01-01").tz_localize("Asia/Ho_Chi_Minh")
# users_in_2025 = stats_df[stats_df['createTime']
#                          >= time_thres]['username'].unique()
# filt_stats_df = stats_df[stats_df['username'].isin(users_in_2025)]

time_thres = pd.to_datetime("2025-01-01").tz_localize("Asia/Ho_Chi_Minh")
filt_stats_df = stats_df[stats_df['createTime'] >= time_thres]
# filt_stats_df = stats_df

print("Number of users: ", filt_stats_df['username'].nunique())

Number of users:  264


### Các thông số thống kê theo từng user về lượt xem và độ tương tác.


In [24]:
# Lọc lại các cột quan trọng liên quan đến user và các chỉ số tương tác của video
df_selected = filt_stats_df[[
    # "author.id",  # ID của user
    "username",  # Tên hiển thị của user
    "views",  # Lượt xem video
    "likes",  # Lượt thích video
    "comments",  # Lượt bình luận video
    "shares",  # Lượt chia sẻ video
    'collects',
    'user.followers',
    'user.likes'
]]

# Kiểm tra dữ liệu bị thiếu
df_selected = df_selected.dropna()

# Tổng hợp dữ liệu theo user
user_stats_df_0 = df_selected.groupby(["username"]).agg(
    total_views=("views", "sum"),
    avg_views=("views", "mean"),
    total_likes=("likes", "sum"),
    avg_likes=("likes", "mean"),
    total_comments=("comments", "sum"),
    total_shares=("shares", "sum"),
    total_collects=("collects", "sum"),
    total_videos=("views", "count"),  # Số lượng video của user
    followers=('user.followers', 'max'),
    likes=('user.likes', 'max'),
).reset_index()

user_stats_df_0 = user_stats_df_0.rename(columns={'username': 'username'})
# Tính Engagement Rate (ER) trung bình
user_stats_df_0["engagement_rate"] = (user_stats_df_0["total_likes"] + user_stats_df_0["total_comments"] +
                                      user_stats_df_0["total_shares"]+user_stats_df_0["total_collects"]) / user_stats_df_0["total_views"]
user_stats_df_0["total_engagement"] = (user_stats_df_0["total_likes"] + user_stats_df_0["total_comments"] +
                                       user_stats_df_0["total_shares"]+user_stats_df_0["total_collects"])
# user_stats_df_0["avg_engagement"] = user_stats_df_0["total_engagement"] / \
#     user_stats_df_0["total_videos"]
# Loại bỏ những user có tổng lượt xem bằng 0 (tránh lỗi chia cho 0)
user_stats_df_0 = user_stats_df_0[user_stats_df_0["total_views"] > 0]
display(user_stats_df_0.sample(5))
# display(user_stats_df_0.describe())

,username,total_views,avg_views,total_likes,avg_likes,total_comments,total_shares,total_collects,total_videos,followers,likes,engagement_rate,total_engagement
92,fancywithtrang_,8939366.0,2.793552e+05,242454.0,7576.687500,5811.0,100013.0,102332.0,32,126800.0,5424.0,0.050407,450610.0
48,bihuyyy,16007000.0,8.424737e+05,588617.0,30979.842105,9684.0,49394.0,49763.0,19,590200.0,7857.0,0.043572,697458.0
124,huynhanhtuan_dienvien,37371300.0,1.010035e+06,876336.0,23684.756757,9513.0,33441.0,44859.0,37,1400000.0,153.0,0.025799,964149.0
192,phamtrang.2401,3849943.0,1.166649e+05,174727.0,5294.757576,758.0,13145.0,17900.0,33,304700.0,48600.0,0.053645,206530.0
171,nghia2302,541559.0,5.415590e+04,10063.0,1006.300000,292.0,5684.0,7459.0,10,114200.0,7375.0,0.043390,23498.0


### Phần trăm video có caption bằng tiếng Việt


In [25]:
# Tổng số video theo từng user
total_videos = stats_df.groupby('username')['textLang'].count()

# Số video có textLang là 'vi'
vi_videos = stats_df[stats_df['textLang'] == 'vi'].groupby('username')[
    'textLang'].count()

# Ghép hai bảng, thay NaN bằng 0 (nếu user không có video tiếng Việt)
percent_vi = (vi_videos / total_videos).fillna(0).reset_index()

# Đổi tên cột
percent_vi.columns = ['username', 'vi_video_percentage']
percent_vi.sort_values('vi_video_percentage', ascending=False)

,username,vi_video_percentage
132,kun_thich_nauan,1.000000
147,lyluclac28,1.000000
136,lenkycungkhoa,1.000000
137,lethanhtuan4999,1.000000
138,lieten91th,1.000000
...,...,...
199,psynojin,0.925000
223,thaothao300491,0.923858
145,lueating,0.920319
114,hoanganhdeptrailam,0.915094


### Khoảng thời gian giữa 2 bài đăng của user mà 75% bài đăng thoả ngưỡng đó.

Tức là 75% các video của có khoảng cách giữa các lần đăng dưới ngưỡng này.


In [26]:
select_df = filt_stats_df
select_df = select_df.sort_values(by=["username", "createTime"])

# Tính khoảng cách thời gian giữa các video liên tiếp của mỗi user
select_df["time_diff"] = select_df.groupby("username")["createTime"].diff()

# Xử lý trường hợp user chỉ có 1 video, không tính được diff
select_df["time_diff"] = select_df["time_diff"].fillna(pd.Timedelta(seconds=0))

# Tính trung bình khoảng cách giữa các video cho mỗi user
frequency_df = select_df.groupby("username")["time_diff"].quantile(
    0.75).dt.total_seconds() / 3600 / 24  # Đổi ra ngày

# Nếu chỉ có 1 video, gán giá trị 'inf' cho 'days_between_videos'
frequency_df = frequency_df.reset_index(name="days_between_videos")

# Xử lý trường hợp user có 1 video
single_video_users = frequency_df[frequency_df["days_between_videos"] == 0]
frequency_df.loc[single_video_users.index, "days_between_videos"] = float(
    'inf')  # Gán giá trị 'inf' cho những người chỉ có 1 video
frequency_df['frequency'] = 1 / frequency_df['days_between_videos']

# Kiểm tra kết quả
frequency_df.head()

,username,days_between_videos,frequency
0,1108ng,1.016840,0.983439
1,3tl.team,3.734821,0.267750
2,_helloanan_,2.096204,0.477053
3,_tidofood,2.000000,0.500000
4,aan_ne8,5.995972,0.166779


### Tính phần trăm cái bài đăng có liên quan đến ẩm thực (dựa trên hashtag)


Danh sách các hashtag, số video tương ứng và số user sử dụng hashtag đó.


In [27]:
# Tách hashtag thành từng dòng riêng biệt
exploded_df = stats_df.explode("hashtags")

# Đếm số lượng video chứa mỗi hashtag
hashtag_video_counts = exploded_df["hashtags"].value_counts().reset_index()
hashtag_video_counts.columns = ["hashtags", "videos_count"]

# Đếm số tài khoản sử dụng mỗi hashtag (số user duy nhất)
hashtag_user_counts = exploded_df.groupby(
    "hashtags")["username"].nunique().reset_index()
hashtag_user_counts.columns = ["hashtags", "users_count"]

# Gộp cả 2 thông tin vào cùng 1 DataFrame
hashtags_df = hashtag_video_counts.merge(hashtag_user_counts, on="hashtags")
hashtags_df["avg_videos_per_user"] = hashtags_df["videos_count"] / \
    hashtags_df["users_count"]
hashtags_df = hashtags_df.sort_values('users_count', ascending=False)
display(hashtags_df)
# px.bar(hashtags_df.head(50), x='users_count', y='hashtags')

,hashtags,videos_count,users_count,avg_videos_per_user
0,ancungtiktok,50798,241,210.780083
1,learnontiktok,31046,192,161.697917
3,xuhuong,21044,169,124.520710
28,,1930,164,11.768293
19,thanhthoiluottet,2633,153,17.209150
...,...,...,...,...
14489,thanheochien,1,1,1.000000
14488,longnuongchao,1,1,1.000000
14487,lamkimchinhanhgon,1,1,1.000000
14486,mientronhaisan,1,1,1.000000


Lọc ra các hashtag có liên quan đến ẩm thực.


In [28]:
food_keywords = [
    "food", "eat", "drink", "chef", "cook", "cooking", "recipe", "baking", "buffet", "steak", "grill", "vegan",
    "anvat",  "anuong",  "angi",  "anngon", "doan", "ancung", 'nauan', 'anchay',
    "douong", "nuocuong",
    "monngon", 'monchay',
    "ancom", "anvat",  "moncom", "buacom", 'comnha', 'buacom', 'comtam', 'comchay',
    "hutieu", "banhmi", "haisan", 'micay', 'mituongden', 'mycay', 'mytuongden',
    "trasua", "nuoc", "caphe", "sinhto", "ruou", 'latte', 'cacao', 'caffe', 'bun', 'banh',
    "amthuc",  "nhahang", "thucdon", "dacsan", "mukbang",
]

# food_keywords = [
#     "food", "eat", "drink", "chef", "cook", "cooking", "recipe", "baking", "buffet", "steak", "grill", "vegan",
#     "amthuc", "an",  "nhahang", "mon", "ngon", "uong", "thucdon", "dacsan", "tiem",
#     # "ancom", "anvat",  "moncom", "buacom", 'comnha', 'buacom',
#     "hutieu", "banhmi", "haisan", "com", 'bun', 'canh',
#     "tra", "sua", "nuoc", "caphe", "sinhto", "ruou", 'latte', 'cacao', 'caffe',
#     'banh',
# ]

food_related_hashtags_df = hashtags_df[hashtags_df["hashtags"].apply(
    lambda x: any(kw in x for kw in food_keywords))]
food_related_hashtags_df = food_related_hashtags_df.sort_values(
    'users_count', ascending=False, ignore_index=True)

# with pd.option_context("display.max_rows", None):
display(food_related_hashtags_df)

,hashtags,videos_count,users_count,avg_videos_per_user
0,ancungtiktok,50798,241,210.780083
1,reviewanngon,22297,127,175.566929
2,mukbang,9896,97,102.020619
3,anvat,2941,95,30.957895
4,food,7627,93,82.010753
...,...,...,...,...
5674,quanngonquan4,1,1,1.000000
5675,lauchanhdayhaisan,1,1,1.000000
5676,lauhaisanchanhday,1,1,1.000000
5677,nhahanghopmat,1,1,1.000000


In [29]:
px.bar(food_related_hashtags_df.head(30),
       y='hashtags', x='users_count', height=700)

In [30]:
# food_related_hashtags_df.to_csv("hashtag_list.csv", index=False)

Tính phần trăm các video có hashtag về ẩm thực của từng user.


In [31]:
# Danh sách các hashtag liên quan đến ẩm thực
# food_related_hashtags = {
#     "reviewanngon", "ancungtiktok", "foodreview", "monngon", "anvat",
#     "foodtour", "eatclean", "buacomgiadinh", "amthuc", "diadiemanuong",
#     "reviewanuongsaigon", "anvatsaigon", "saigonfood"
# }

# Bước 1: Lấy ra 100 hashtag ẩm thực có nhiều video nhất
food_related_hashtags = food_related_hashtags_df.head(50)[
    'hashtags'].to_list()
# Bước 2: Tạo cột mới để đánh dấu video có chứa ít nhất một hashtag liên quan đến ẩm thực
filt_stats_df["is_food_related"] = filt_stats_df["hashtags"].apply(
    lambda tags: any(tag in food_related_hashtags for tag in tags)
)

# Bước 3: Tính tổng số video & số video liên quan đến ẩm thực của mỗi user
food_user_df = filt_stats_df.groupby("username").agg(
    total_videos=("username", "count"),
    food_related_videos=("is_food_related", "sum")
).reset_index()

# Bước 4: Tính phần trăm số video ẩm thực
food_user_df["food_video_percentage"] = (
    food_user_df["food_related_videos"] / food_user_df["total_videos"])

food_user_df = food_user_df.rename(columns={"username": "username"})
food_user_df.head()

,username,total_videos,food_related_videos,food_video_percentage
0,1108ng,67,67,1.000000
1,3tl.team,6,6,1.000000
2,_helloanan_,37,37,1.000000
3,_tidofood,46,46,1.000000
4,aan_ne8,9,8,0.888889


Merge với `user_stats_df_0`.


In [32]:
user_stats_df = user_stats_df_0.merge(percent_vi, on='username').merge(frequency_df, on="username").merge(
    food_user_df, on=['username', 'total_videos'])
user_stats_df = user_stats_df[user_stats_df['frequency'] != 0]
user_stats_df.head()

display(user_stats_df.sample(5))

,username,total_views,avg_views,total_likes,avg_likes,total_comments,total_shares,total_collects,total_videos,followers,likes,engagement_rate,total_engagement,vi_video_percentage,days_between_videos,frequency,food_related_videos,food_video_percentage
152,mhon.vbep,641507.0,42767.133333,17076.0,1138.400000,72.0,1705.0,5754.0,15,22300.0,3823.0,0.038358,24607.0,1.000000,6.079288,0.164493,15,1.000000
60,cholondowntown,9622232.0,165900.551724,386170.0,6658.103448,5422.0,41465.0,48972.0,58,363700.0,4905.0,0.050095,482029.0,0.996364,1.881942,0.531366,44,0.758621
96,gauvlogs1997,20427000.0,334868.852459,325770.0,5340.491803,2891.0,6873.0,12700.0,61,329900.0,579.0,0.017048,348234.0,0.986900,1.064861,0.939090,59,0.967213
212,spicykim9386,39058000.0,685228.070175,1005079.0,17632.964912,9730.0,50191.0,30263.0,57,2900000.0,1382.0,0.028042,1095263.0,0.993289,1.753414,0.570316,55,0.964912
178,nguoidean_khongdetinh,3312373.0,174335.421053,109843.0,5781.210526,2464.0,33844.0,35437.0,19,71600.0,3304.0,0.054821,181588.0,0.975155,3.927807,0.254595,19,1.000000


### Lọc user theo các ngưỡng


#### Visualization


In [33]:
px.histogram(user_stats_df, x='vi_video_percentage',
             title="Phân phối User theo Phần trăm video có caption tiếng Việt", width=600)

In [34]:
px.histogram(user_stats_df, x='food_video_percentage',
             title="Phân phối User theo Phần trăm video liên quan đến Ẩm thực", nbins=20)

In [35]:
px.histogram(user_stats_df, x='days_between_videos',
             title="Phân phối User theo Khoảng thời gian giữa các bài đăng")

In [36]:
px.histogram(user_stats_df, x='engagement_rate',
             title="Phân phối User theo Tỷ lệ tương tác")

In [37]:
px.histogram(user_stats_df, x='avg_views',
             title="Phân phối User theo Lượt xem trung bình")

In [38]:
px.histogram(user_stats_df, x='followers',
             title="Phân phối User theo Followers")

In [39]:
px.histogram(user_stats_df, x='likes',
             title="Phân phối User theo Likes")

#### Các tiêu chuẩn lọc user


In [40]:
print(user_stats_df.shape)
user_stats_df[['username', 'avg_views', 'total_videos', 'followers',
               'engagement_rate', 'frequency',
               'food_video_percentage', 'vi_video_percentage']].describe()

(264, 18)


,avg_views,total_videos,followers,engagement_rate,frequency,food_video_percentage,vi_video_percentage
count,2.640000e+02,264.000000,2.640000e+02,264.000000,264.000000,264.000000,264.000000
mean,3.712888e+05,44.734848,3.364726e+05,0.037695,0.529313,0.930200,0.990727
std,4.329574e+05,34.037970,4.995458e+05,0.016577,0.367324,0.115659,0.016531
min,1.293000e+03,2.000000,3.927000e+03,0.008474,0.037281,0.000000,0.902077
25%,9.125341e+04,22.000000,6.442500e+04,0.025235,0.284579,0.894630,0.990377
50%,2.103005e+05,38.000000,1.469500e+05,0.034959,0.478862,0.966365,0.997174
75%,4.666074e+05,61.000000,3.458750e+05,0.047538,0.808371,1.000000,1.000000
max,2.254700e+06,259.000000,3.000000e+06,0.111330,3.050578,1.000000,1.000000


**Lọc theo Phần trăm video có caption tiếng Việt và Phần trăm video liên quan sử dụng hashtag ẩm thực của user.**


In [41]:
print(user_stats_df.shape)

# Bước 0: Lọc theo vi_video_percentage
min_vi_perc_threshold = 0.90
users_by_threshold = user_stats_df[
    user_stats_df["vi_video_percentage"] >= min_vi_perc_threshold
]
print(f"Min Vietnamese Video Percentage Threshold: {min_vi_perc_threshold}")
print(users_by_threshold.shape)

# Bước 1: Lọc theo food_video_percentage
# min_food_perc_threshold = users_by_threshold["food_video_percentage"].quantile(
#     0.6)
min_food_perc_threshold = 0.75
users_by_threshold = users_by_threshold[
    users_by_threshold["food_video_percentage"] >= min_food_perc_threshold
]
print(f"Min Food Video Percentage Threshold: {min_food_perc_threshold}")
print(users_by_threshold.shape)

# # Bước 2: Lọc tiếp theo avg_views
# min_avg_views_threshold = users_by_threshold["avg_views"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["avg_views"] >= min_avg_views_threshold
# ]
# print(f"Min Average Views Threshold: {min_avg_views_threshold}")
# print(users_by_threshold.shape)

# # Bước 3: Lọc tiếp theo engagement_rate
# min_eng_rate_threshold = users_by_threshold["engagement_rate"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["engagement_rate"] >= min_eng_rate_threshold
# ]
# print(f"Min Engagement Rate Threshold: {min_eng_rate_threshold}")
# print(users_by_threshold.shape)

# # Bước 4: Lọc cuối cùng theo frequency
# min_freq_threshold = users_by_threshold["frequency"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["frequency"] >= min_freq_threshold
# ]
# print(f"Min Frequency Threshold: {min_freq_threshold}")
# print(users_by_threshold.shape)

users_by_threshold.sort_values(by="total_videos", ascending=True)

(264, 18)
Min Vietnamese Video Percentage Threshold: 0.9
(264, 18)
Min Food Video Percentage Threshold: 0.75
(257, 18)


,username,total_views,avg_views,total_likes,avg_likes,total_comments,total_shares,total_collects,total_videos,followers,likes,engagement_rate,total_engagement,vi_video_percentage,days_between_videos,frequency,food_related_videos,food_video_percentage
218,tebetravelmade,3733.0,1.866500e+03,83.0,41.500000,0.0,13.0,20.0,2,55100.0,398.0,0.031074,116.0,0.983936,7.479332,0.133702,2,1.000000
210,seonguk_vn,348400.0,1.742000e+05,13050.0,6525.000000,141.0,81.0,180.0,2,310500.0,2449.0,0.038611,13452.0,1.000000,2.166944,0.461479,2,1.000000
35,be_mup1,2487700.0,1.243850e+06,74200.0,37100.000000,2401.0,6418.0,3815.0,2,23900.0,862.0,0.034905,86834.0,1.000000,15.751172,0.063487,2,1.000000
116,homnay_tuiangi,165500.0,8.275000e+04,2075.0,1037.500000,48.0,312.0,158.0,2,146900.0,19800.0,0.015668,2593.0,1.000000,24.858767,0.040227,2,1.000000
74,datvaingang,4501500.0,1.500500e+06,208363.0,69454.333333,1326.0,5285.0,9272.0,3,154300.0,488.0,0.049816,224246.0,1.000000,5.964705,0.167653,3,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,dung.thch.n,47800900.0,3.824072e+05,1296448.0,10371.584000,11757.0,82631.0,54204.0,125,148300.0,416.0,0.030230,1445040.0,0.996117,0.678241,1.474403,122,0.976000
257,wottueat,34871765.0,2.165948e+05,648678.0,4029.055901,12359.0,106688.0,35061.0,161,37300.0,2622.0,0.023021,802786.0,0.994266,0.690764,1.447673,161,1.000000
232,tinhtungtangvn,45901933.0,2.638042e+05,1419104.0,8155.770115,34194.0,643654.0,354676.0,174,295300.0,5455.0,0.053410,2451628.0,1.000000,0.650330,1.537681,170,0.977011
246,tunauan68,76556779.0,3.112064e+05,1902577.0,7734.052846,121802.0,53384.0,131475.0,246,278300.0,18.0,0.028858,2209238.0,0.991946,0.593808,1.684046,235,0.955285


In [42]:
video_info_df[video_info_df['author.uniqueId'].isin(
    users_by_threshold['username'])].shape

(69306, 80)

Lưu danh sách user cuối cùng để crawl lại tất cả cho đồng bộ.


In [43]:
def write_list_to_file(file_path, data_list):
    try:
        with open(file_path, "w", encoding="utf-8") as file:
            for item in data_list:
                file.write(str(item) + "\n")
        print(f"Successfully wrote {len(data_list)} items to {file_path}")
    except Exception as e:
        print(f"Error writing to file: {e}")


write_list_to_file('data/lists/final_users_list.txt',
                   list(users_by_threshold['username'].unique()))

Successfully wrote 257 items to data/lists/final_users_list.txt


### Lọc bằng cách tính điểm


In [44]:

# def compute_score(df, columns_weights):
#     """
#     Tính điểm tổng hợp cho DataFrame user_stats_df với các cột và trọng số linh hoạt.

#     Công thức tính điểm:
#       score = sum(weight_i * norm_column_i)
#     """
#     df_copy = df.copy()

#     for col in columns_weights.keys():
#         df_copy[col] = df_copy[col].fillna(0).clip(lower=0)

#     max_values = {col: df_copy[col].max(
#     ) or 1 for col in columns_weights.keys()}

#     for col in columns_weights.keys():
#         df_copy[f"norm_{col}"] = df_copy[col] / max_values[col]

#     df_copy["score"] = sum(
#         columns_weights[col] * df_copy[f"norm_{col}"]*100
#         for col in columns_weights.keys()
#     )

#     return df_copy

In [45]:

# # Ví dụ sử dụng:
# columns_weights = {
#     "food_video_percentage": 0.35,
#     "engagement_rate": 0.05,
#     "avg_views": 0.35,
#     "frequency": 0.1,
#     "followers": 0.35
# }
# # columns_weights = {
# #     "avg_views": 0.35,
# #     "engagement_rate": 0.25,
# #     "followers": 0.35,
# #     "frequency": 0.05
# # }

# # Tính điểm cho DataFrame
# # users_by_score = compute_score(
# #     user_stats_df[user_stats_df['vi_video_percentage'] >= 0.8], columns_weights)
# # users_by_score = compute_score(users_by_threshold, columns_weights)
# users_by_score = compute_score(
#     users_by_threshold, columns_weights)

# # In kết quả
# users_by_score.sort_values('score', ascending=False)

# # Sau đó bạn có thể:
# users_by_score = users_by_score.sort_values(
#     "score", ascending=False, ignore_index=True)

# uesr_by_score_top = users_by_score.head(len(users_by_threshold))
# display(users_by_score[["username", "score"]].merge(user_stats_df[[
#         'followers', 'username', 'avg_views', 'frequency', 'engagement_rate']]))

In [46]:
# user_stats_df.columns

In [47]:
# import json


# def calculate_entropy_weights(df, columns_to_maximize):
#     """
#     Tính trọng số (weights) cho mỗi cột trong DataFrame df dựa trên phương pháp Entropy Weighting.
#     """
#     # 1) Chuẩn hóa dữ liệu về [0..1]
#     df_normalized = df[columns_to_maximize].copy()
#     for col in columns_to_maximize:
#         df_normalized[col] = (df[col] - df[col].min()) / \
#             (df[col].max() - df[col].min())

#     # 2) Tính entropy cho mỗi cột
#     n = len(df)  # số lượng user
#     p = df_normalized / df_normalized.sum(axis=0)  # xác suất
#     entropy = - (p * np.log(p + 1e-9)).sum(axis=0) / np.log(n)  # tránh log(0)

#     # 3) Tính trọng số (weights) cho mỗi cột
#     weights = (1 - entropy) / (1 - entropy).sum()

#     # Trả về DataFrame với trọng số
#     weights_df = pd.DataFrame(
#         weights, columns=['weight'], index=columns_to_maximize)

#     return weights_df


# columns_to_maximize = ['total_views', 'avg_views', 'total_videos', 'total_engagement', 'followers',
#                        # 'total_likes', 'avg_likes', 'total_comments', 'total_shares', 'total_collects', 'cluster', 'days_between_videos',
#                        'engagement_rate', 'frequency', 'food_related_videos', 'food_video_percentage']


# # Tính trọng số cho mỗi cột
# weights_df = calculate_entropy_weights(
#     user_stats_df, columns_to_maximize).sort_values("weight")
# display(weights_df)
# # Xem kết quả
# columns_weights = json.loads(weights_df.to_json())['weight']

In [48]:
# # Tính điểm cho DataFrame
# df_with_scores = compute_score(user_stats_df, columns_weights)

# # In kết quả
# df_with_scores.sort_values('score', ascending=False)

# # Sau đó bạn có thể:
# user_stats_df_scored = df_with_scores.sort_values(
#     "score", ascending=False)
# user_stats_df_scored[["username", "score"]].head(100)  # Xem top 20

In [49]:
# from sklearn.cluster import KMeans
# columns_to_consider = ['total_views', 'avg_views', 'total_likes', 'avg_likes',
#                        'total_comments', 'total_shares', 'total_collects',
#                        'total_videos', 'engagement_rate', 'days_between_videos',
#                        'food_related_videos', 'food_video_percentage', 'frequency']
# # Sử dụng các cột để phân nhóm (có thể chuẩn hóa dữ liệu nếu cần)
# X = user_stats_df[columns_to_consider]

# # Áp dụng KMeans (giả sử chia thành 3 nhóm)
# kmeans = KMeans(n_clusters=3)
# user_stats_df['cluster'] = kmeans.fit_predict(X)

# # Lấy các centroid của mỗi nhóm
# centroids = kmeans.cluster_centers_

# # Các cột quan trọng có thể được xác định từ các giá trị lớn trong centroids
# centroid_df = pd.DataFrame(centroids, columns=columns_to_consider)
# centroid_df

In [50]:
# def get_pareto_front(df, columns_maximize):
#     """
#     Tìm Pareto front trong DataFrame df, coi tất cả cột trong columns_maximize
#     đều là "càng lớn càng tốt".

#     Trả về sub-DataFrame gồm các hàng không bị chi phối bởi bất kỳ hàng nào khác.
#     """
#     dominated = set()
#     n = len(df)

#     for i in range(n):
#         if i in dominated:
#             # Hàng này đã bị chi phối, không cần check nữa
#             continue

#         # Kiểm tra xem có hàng j nào chi phối i không
#         for j in range(n):
#             if i == j:
#                 continue
#             # Nếu j >= i trên toàn bộ columns,
#             # và j > i trên ít nhất 1 cột => i bị chi phối
#             if (df.iloc[j][columns_maximize] >= df.iloc[i][columns_maximize]).all() \
#                and (df.iloc[j][columns_maximize] > df.iloc[i][columns_maximize]).any():
#                 dominated.add(i)
#                 break

#     # Các hàng không bị chi phối -> Pareto front
#     indices_front = [idx for idx in range(n) if idx not in dominated]
#     return df.iloc[indices_front].copy()

In [51]:
# # Tìm Pareto front
# pareto_df = get_pareto_front(user_stats_df, columns_to_maximize)

# print(f"Số user trên Pareto front: {len(pareto_df)}")
# pareto_df.head()

In [52]:
# pareto_df.columns